# Data Analysis and Visualisation &mdash; Lab 01

## Problem 1: How has Linux adoption as a desktop operating system changed up to 2025?

### 1. Objective

The objective of this study is to examine the trend of Linux adoption as a desktop operating system up to 2025. By collecting data from public sources (*i.e.* StatCounter, Steam Hardware Survey, DistroWatch), we aim to observe whether Linux usage among desktop users is increasing, or remaining stable over time.

### 2. Variables of Interest

| Variable | Description | Type | Source |
| --- | --- | --- | --- |
| `date` | Month and year of observation | Datetime | All sources |
| `region` | Geographic region or country | Categorical | StatCounter |
| `linux_share` | % of desktop users using Linux | Numeric | StatCounter |
| `win_share` | % of desktop users using Windows | Numeric | StatCounter |
| `mac_share` | % of desktop users using macOS | Numeric | StatCounter |
| `steam_linux_share` | % of Steam users on Linux | Numeric | Steam Hardware Survey |
| `distro_hits` | Page hits per day for each Linux distro (popularity indicator) | Numeric | DistroWatch |

In [1]:
import urllib.parse
import pandas as pd
import pycountry

def get_statcounter_os_share(start = "2023-01", end = "2025-10", country = "ww"):
    region = pycountry.countries.get(alpha_2=country).name if country != "ww" else "Worldwide"
    region_enc = urllib.parse.quote(region)
    
    csv_url = (
        f"https://gs.statcounter.com/os-market-share/desktop/chart.php"
        f"?device=Desktop"
        f"&device_hidden=desktop"
        f"&statType=Operating%20System"
        f"&statType_hidden=os_combined"
        f"&region={region_enc}"
        f"&region_hidden={country}"
        f"&granularity=monthly"
        f"&fromInt={start.replace('-', '')}"
        f"&fromMonthYear={start}"
        f"&toInt={end.replace('-', '')}"
        f"&toMonthYear={end}"
        f"&csv=1"
    )

    df = pd.read_csv(csv_url)
    df["Region"] = region
    return df

statcounter_ww = get_statcounter_os_share()
statcounter_vn = get_statcounter_os_share(country="VN")

statcounter_ww.to_csv("StatCounter_Worldwide.csv", index=False)
statcounter_vn.to_csv("StatCounter_Viet+Nam.csv", index=False)

In [4]:
steam_hw_survey = pd.read_csv("https://raw.githubusercontent.com/jdegene/steamHWsurvey/refs/heads/master/shs_platform.csv")
steam_hw_survey = steam_hw_survey[(steam_hw_survey["date"] >= "2023-01-01") & (steam_hw_survey["date"] <= "2025-10-01")]

platform_versions = steam_hw_survey[steam_hw_survey["category"].isin(["Windows Version", "OSX Version", "Linux Version"])]

platform_versions.to_csv("Steam+HW+Survey_Platform+Versions.csv", index=False)

In [5]:
def get_distrowatch_phr():
    url = "https://distrowatch.com/dwres.php?resource=popularity"
    df = pd.DataFrame()
    
    for table in pd.read_html(url):
        cols = table.columns.astype(str)
        
        if any("Last" in col for col in cols):
            month_range = cols[0].split(" ")[1]
            
            table.columns = ["ranking", "distro", "hits_per_day"]
            table["month_range"] = month_range

            df = pd.concat([df, table], ignore_index=True)
            
    return df

distrowatch_phr = get_distrowatch_phr()
distrowatch_phr.to_csv("DistroWatch_HPD.csv", index=False)

## Problem 2: Does having access to e-books anywhere affect the usage and sales of physical books?

### 1. Objective

The objective of this study is to study how the growing availability and convenience of e-books affect the usage and sales of physical books by understanding whether digital reading trends are reducing physical book purchases or merely expanding overall reading habits.

### 2. Variables of Interest

| Variable | Description | Type | Source |
| --- | --- | --- | --- |
| `year` | Year of record | Numeric | |
| `region` | Geographic region or country | Categorical | |
| `ebook_sales` | Revenue or sale units of e-books | Numeric | |
| `print_sales` | Revenue or sale units of physical books | Numeric | |
| `total_sales` | Total revenue or sale units | Numeric | |
| `ebook_share` | Proportion of e-book sales over total sales | Numeric | |
| `avg_ebook_price` | Average approximated e-book price | Numeric | |
| `avg_print_price` | Average approximated physical book price | Numeric | |

,ASIN,GROUP,FORMAT,TITLE,AUTHOR,PUBLISHER
0,1250150183,book,hardcover,The Swamp: Washington's Murky Pool of Corrupti...,Eric Bolling,St. Martin's Press
1,0778319997,book,hardcover,"Rise and Shine, Benedict Stone: A Novel",Phaedra Patrick,Park Row Books
2,1608322564,book,hardcover,Sell or Be Sold: How to Get Your Way in Busine...,Grant Cardone,Greenleaf Book Group Press
3,0310325331,book,hardcover,Christian Apologetics: An Anthology of Primary...,"Khaldoun A. Sweis, Chad V. Meister",Zondervan
4,0312616295,book,hardcover,Gravity: How the Weakest Force in the Universe...,Brian Clegg,St. Martin's Press
